# Traveling salesman problem

Find the closed tour that visits all ten cities and has the minimum total length. Distances between pairs of cities are given in the table below:

|         |  ATL  |  ORD  |  DEN  |  IAH  |  LAX  |  MIA  |  JFK  |  SFO  |  SEA  |  DCA  |
|--------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|
|**ATL**  |    0  |  587  | 1212  |  701  | 1936  |  604  |  748  | 2139  | 2182  |  543  |
|**ORD**  |  587  |    0  |  920  |  940  | 1745  | 1188  |  713  | 1858  | 1737  |  597  |
|**DEN**  | 1212  |  920  |    0  |  879  |  831  | 1726  | 1631  |  949  | 1021  | 1494  |
|**IAH**  |  701  |  940  |  879  |    0  | 1379  |  968  | 1420  | 1645  | 1891  | 1220  |
|**LAX**  | 1936  | 1745  |  831  | 1379  |    0  | 2339  | 2451  |  347  |  959  | 2300  |
|**MIA**  |  604  | 1188  | 1726  |  968  | 2339  |    0  | 1092  | 2594  | 2734  |  923  |
|**JFK**  |  748  |  713  | 1631  | 1420  | 2451  | 1092  |    0  | 2571  | 2408  |  205  |
|**SFO**  | 2139  | 1858  |  949  | 1645  |  347  | 2594  | 2571  |    0  |  678  | 2442  |
|**SEA**  | 2182  | 1737  | 1021  | 1891  |  959  | 2734  | 2408  |  678  |    0  | 2329  |
|**DCA**  |  543  |  597  | 1494  | 1220  | 2300  |  923  |  205  | 2442  | 2329  |    0  |

In [ ]:
# Define the problem data (cities and costs)
# Run this, then run the final block of this notebook to load the helper functions before continuing
using JuMP, NamedArrays, Gurobi

cities = [:ATL, :ORD, :DEN, :IAH, :LAX, :MIA, :JFK, :SFO, :SEA, :DCA]

distances = [     0   587  1212   701  1936   604   748  2139  2182   543
                587     0   920   940  1745  1188   713  1858  1737   597
               1212   920     0   879   831  1726  1631   949  1021  1494
                701   940   879     0  1379   968  1420  1645  1891  1220
               1936  1745   831  1379     0  2339  2451   347   959  2300
                604  1188  1726   968  2339     0  1092  2594  2734   923
                748   713  1631  1420  2451  1092     0  2571  2408   205
               2139  1858   949  1645   347  2594  2571     0   678  2442
               2182  1737  1021  1891   959  2734  2408   678     0  2329
                543   597  1494  1220  2300   923   205  2442  2329     0  ]

c = NamedArray(distances,(cities,cities))
N = size(distances,1);

In [ ]:
function printTspSol(x)
    for i in 1:N 
        for j in 1:N 
            if x[i,j] > 0.5
                println(cities[i], " -> ", cities[j])
            end 
        end 
    end 
end

## Solve TSP using min-cost flow relaxation

In [ ]:
# solve the simplest version (min-cost flow version; it's an assignment problem)


m = Model(Gurobi.Optimizer)
# set_optimizer_attribute(m, "OutputFlag", 0)
set_silent(m)

@variable(m, 0 <= x[cities,cities] <= 1)                                 # LP relaxation
@constraint(m, c1[j in cities], sum( x[i,j] for i in cities ) == 1)    # exacly one edge out of each node
@constraint(m, c2[i in cities], sum( x[i,j] for j in cities ) == 1)    # exactly one edge into each node
@constraint(m, c3[i in cities], x[i,i] == 0 )                       # no self-loops
@objective(m, Min, sum( x[i,j]*c[i,j] for i in cities, j in cities ))  # minimize total cost
optimize!(m)

# pretty print the solution
xx = value.(x)
sol = NamedArray(zeros(Int,N,N),(cities,cities))
for i in cities
    for j in cities
        sol[i,j] = Int(xx[i,j])
    end
end

printTspSol(sol)


## Solve TSP using adaptive subtour elimination

### Helper functions

In [ ]:
# HELPER FUNCTION: returns the cycle containing the city START.
function getSubtour(x,start)
    subtour = [start]
    while true
        j = subtour[end]
        for k in cities
            if x[k,j] == 1
                push!(subtour,k)
                break
            end
        end
        if subtour[end] == start
            break
        end
    end
    return subtour
end

# HELPER FUNCTION: returns a list of all cycles
function getAllSubtours(x)
    nodesRemaining = cities
    subtours = []
    while length(nodesRemaining) > 0
        subtour = getSubtour(x,nodesRemaining[1])
        push!(subtours, subtour)
        nodesRemaining = setdiff(nodesRemaining,subtour)
    end
    return subtours
end


### Optimization problem

In [ ]:
m = Model(Gurobi.Optimizer)
# set_optimizer_attribute(m, "OutputFlag", 0)
set_silent(m)

@variable(m, x[cities,cities], Bin)                                       # must formulate as IP this time
@constraint(m, c1[j in cities], sum( x[i,j] for i in cities ) == 1)     # one out-edge
@constraint(m, c2[i in cities], sum( x[i,j] for j in cities ) == 1)     # one in-edge
@constraint(m, c3[i in cities], x[i,i] == 0 )                        # no self-loops
@objective(m, Min, sum( x[i,j]*c[i,j] for i in cities, j in cities ))   # minimize total cost

sols = []

for iters = 1:30
    optimize!(m)
    println("Tour length: ", objective_value(m))
    xx = value.(x)
    push!(sols,xx)
    subtours = getAllSubtours(xx)  # get all the subtours
    display(subtours)
    len = length(subtours)
    if len == 1                    # solution is just a single tour!
        println("SOLVED!")
        break
    else
        for subtour in subtours
            L = length(subtour)
            @constraint(m, sum( x[subtour[k+1],subtour[k]] for k = 1:L-1 ) <= L-2)
            @constraint(m, sum( x[subtour[k],subtour[k+1]] for k = 1:L-1 ) <= L-2)
        end
    end
end



## Miller-Tucker-Zemlin formulation

In [ ]:
m = Model(Gurobi.Optimizer)
# set_optimizer_attribute(m, "OutputFlag", 0)
set_silent(m)

@variable(m, x[cities,cities], Bin)                                      # must formulate as IP this time
@constraint(m, c1[j in cities], sum( x[i,j] for i in cities ) == 1)      # one out-edge
@constraint(m, c2[i in cities], sum( x[i,j] for j in cities ) == 1)      # one in-edge
@constraint(m, c3[i in cities], x[i,i] == 0 )                            # no self-loops
@objective(m, Min, sum( x[i,j]*c[i,j] for i in cities, j in cities ))   # minimize total cost
                                    
# MTZ variables and constraints
@variable(m, u[cities])
@constraint(m, c4[i in cities, j in cities[2:end]], u[i] - u[j] + N*x[i,j] <= N-1 )

optimize!(m)
xx = value.(x)
subtours = getAllSubtours(xx)  # get cycle containing Atlanta
display(subtours)
println("Tour length: ", objective_value(m))